In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import sklearn
import pickle
from sklearn import pipeline
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn import ensemble
from sklearn import neural_network
from sklearn import decomposition

In [5]:
raw_features_path = "/content/drive/MyDrive/moody_playlist_data/raw_features.csv"
raw_features = pd.read_csv(raw_features_path)

In [6]:
basic_features_path = "/content/drive/MyDrive/moody_playlist_data/features.csv"
basic_features = pd.read_csv(basic_features_path)

In [7]:
combined_raw = pd.concat([basic_features, raw_features], axis = 1)
combined_raw['song_check'] = combined_raw['title'] + ' - ' + combined_raw['artist']
if combined_raw['song'].equals(combined_raw['song_check']):
    print ("Raw features match.")
    combined_raw = combined_raw.drop(columns = ['song', 'song_check'])
else:
    print ("Raw features DO NOT match.")

Raw features match.


In [8]:
combined_raw

,title,artist,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid,primary,secondary,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,...,99960,99961,99962,99963,99964,99965,99966,99967,99968,99969,99970,99971,99972,99973,99974,99975,99976,99977,99978,99979,99980,99981,99982,99983,99984,99985,99986,99987,99988,99989,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
0,a new adventure,takanashi yasuharu,117.453835,-0.010635,0.082014,3.164051,0.186062,1,6,0.133759,0.116821,0.142548,0.124207,0.009430,0.067474,0.181488,0.219666,0.106842,-0.174561,-0.307129,-0.324890,-0.083984,-0.002960,0.092987,-0.013733,-0.183990,-0.217865,-0.216522,-0.159241,-0.160767,0.085083,0.043732,-0.120087,-0.259277,-0.099213,-0.188293,0.012268,0.029846,0.010315,-0.001953,...,0.155273,0.184082,0.053253,0.086792,0.202667,0.119232,-0.093842,-0.065155,0.147888,0.196228,0.130829,-0.006805,0.082336,-0.111420,-0.269745,-0.052246,-0.124756,0.009796,-0.174103,-0.352661,-0.272919,-0.187561,-0.075317,-0.010376,-0.072021,-0.007263,-0.077972,0.000153,0.033020,-0.133118,-0.080841,-0.034271,-0.182251,0.017303,0.132935,0.306610,0.148407,-0.014404,0.183716,0.136566
1,a oh,super junior,129.199219,-0.005782,0.078039,3.213386,0.179459,3,2,0.036011,0.041168,0.049866,0.052704,0.049591,0.041840,0.034576,0.038391,0.046722,0.044220,0.036621,0.046051,0.067596,0.075500,0.066803,0.056213,0.044464,0.037018,0.043945,0.059082,0.066772,0.060669,0.048828,0.041412,0.026337,-0.007507,-0.038635,-0.045898,-0.034515,-0.024139,-0.036530,...,0.010681,0.027618,0.029266,0.003479,-0.031036,-0.017517,-0.049591,-0.036987,0.035645,0.022614,-0.110046,-0.171906,-0.134247,-0.031189,0.015625,-0.045563,-0.089325,0.014465,0.117493,0.098602,0.046997,-0.011322,-0.070007,-0.050446,-0.013580,-0.082764,-0.140228,-0.126617,-0.130219,-0.094116,-0.007111,-0.030304,-0.008392,0.059509,0.092468,0.025116,0.016785,0.041809,0.071136,0.130554
2,a thousand years,christina perri,92.285156,-0.016321,0.044617,3.217903,0.122203,6,5,0.055420,0.006805,-0.044891,-0.095825,-0.134277,-0.156830,-0.174744,-0.179108,-0.175995,-0.170349,-0.161591,-0.155548,-0.160583,-0.165436,-0.157532,-0.139526,-0.116089,-0.094452,-0.081116,-0.073120,-0.055878,-0.020569,0.023346,0.065613,0.107117,0.149323,0.186401,0.205780,0.205505,0.196716,0.188049,...,-0.069733,-0.063721,-0.055878,-0.048798,-0.043335,-0.038116,-0.034332,-0.029358,-0.024780,-0.020721,-0.021881,-0.022247,-0.022034,-0.022125,-0.025269,-0.027222,-0.027863,-0.029541,-0.029785,-0.026886,-0.022827,-0.021790,-0.023651,-0.029633,-0.037109,-0.046234,-0.053772,-0.059326,-0.061462,-0.061310,-0.058014,-0.056671,-0.054871,-0.055237,-0.053772,-0.048981,-0.044281,-0.034943,-0.029633,-0.027527
3,adore u,seventeen,103.359375,-0.007745,0.058125,3.142738,0.145660,3,2,-0.197845,-0.249542,-0.231934,-0.196228,-0.125092,0.024902,0.097107,0.132629,0.159119,0.110992,0.063507,0.015350,-0.012024,0.025635,0.000549,-0.000122,0.010620,-0.079895,-0.089844,-0.075348,-0.156830,-0.126526,-0.172668,-0.194214,-0.089844,0.004517,-0.005066,0.084961,-0.028473,-0.156372,-0.102722,...,0.045868,0.125397,-0.179932,0.108307,-0.221252,0.195251,0.000946,0.014099,-0.194092,-0.185425,0.241699,0.025299,-0.042542,-0.114655,0.087463,0.014252,0.156952,-0.078156,-0.177704,0.092926,-0.015839,0.064117,-0.130524,-0.045563,0.039703,0.136902,0.011871,0.015930,-0.123718,0.054565,-0.056244,0.125000,-0.037994,0.080261,-0.024231,-0.150635,-0.049500,0.043701,-0.050781,0.124878
4,after rain,aimer,117.453835,-0.010324,0.068620,3.211794,0.147982,2,5,0.088409,0.230469,0.270050,0.196136,0.080658,0.001160,-0.000366,0.056885,0.119629,0.138916,0.092010,0.000275,-0.075134,-0.084290,-0.015533,0.078308,0.135834,0.127350,0.108093,0.130798,0.181519,0.209595,0.181915,0.114105,0.039032,-0.008575,-0.011169,0.006165,0.001770,-0.036346,-0.064331,...,0.002228,0.059570,0.094330,0.083130,0.021271,-0.047668,-0.078918,-0.084808,-0.104095,-0.146606,-0.196167,-0.213898,-0.195343,-0.137665,-0.058563,0.007965,

In [9]:
def build_xy(features):
    x = features.iloc[:, 9:]
    y = features[['primary', 'secondary']]
    y['combined'] = y['primary'].astype(str) + y['secondary'].astype(str)
    return (x, y)

In [17]:
# Multi-Layer Perceptron with raw features
x, y = build_xy(combined_raw)
mlp_model = neural_network.MLPClassifier()
scaler = preprocessing.MinMaxScaler()
pca = decomposition.PCA(n_components = 0.95, svd_solver = "full")
pipe = pipeline.Pipeline(steps = [('scaler', scaler), ('pca', pca), ('mlp', mlp_model)])
param_grid = {
    'mlp__hidden_layer_sizes': [(2000,)],
}
model = model_selection.GridSearchCV(pipe, param_grid, cv = 5)
# model.fit(x, y['primary'])
# print ("***SVM Best Parameters***")
# print (model.best_params_)
accuracies = model_selection.cross_val_score(model, x, y['primary'], cv = 5)
print("Average accuracy:", np.mean(accuracies))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarn

Average accuracy: 0.215


In [11]:
print("Average accuracy:", np.mean(accuracies))

Average accuracy: 0.21000000000000002
